# 2. Update data daily

In [15]:
import pickle
import pandas as pd
import hopsworks
import os

from functions.TomTomAPI import get_traffic_map_from_grid
from functions.HolidaysnWeather import get_weather, is_holiday

In [18]:
coordinates = 59.34318, 18.05141 # Stockholm near Odenplan
zoom = 20

# Get the API key from GitHub Secrets
HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')
TOMTOM_API_KEY = os.getenv('TOMTOM_API_KEY')
TOMTOM_API_KEY2 = os.getenv('TOMTOM_API_KEY2')
TOMTOM_API_KEY3 = os.getenv('TOMTOM_API_KEY3')
CALENDAR_API_KEY = os.getenv('CALENDAR_API_KEY')

# with open('hopsworks-api-key.txt', 'r') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

TOMTOM_keys = [TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3]

Connection closed.
2024-12-18 21:45:59,441 INFO: Initializing external client
2024-12-18 21:45:59,443 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-18 21:46:00,595 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170582
2024-12-18 21:46:01,776 INFO: Closing external client and cleaning up certificates.
2024-12-18 21:46:01,780 INFO: Initializing external client


ExternalClientError: host cannot be of type NoneType, host is a non-optional argument to connect to hopsworks from an external environment.

### 2.1. Collect traffic data

In [8]:
pickle_path = 'notebooks/variables/grid.pickle'

# Getting grid variable from memory
with open(pickle_path, 'rb') as file:
    # Deserialize and retrieve grid from pickle file
    grid = pickle.load(file)

In [9]:
# Collecting traffic flow from grid
success = False
i = 0
while i < len(TOMTOM_keys) and not success:
    try:
        traffic_map = get_traffic_map_from_grid(TOMTOM_keys[i],grid, zoom = zoom)
        success = True
    except:
        print('Failed with key:', i)
        i += 1

Failed with key: 0
Failed with key: 1
num of requests: 288


In [10]:
from shapely.geometry import LineString
import datetime

traffic_df = pd.DataFrame(traffic_map)
traffic_df['coordinates'] = traffic_df['coordinates'].apply(lambda x: LineString([(point['longitude'], point['latitude']) for point in x['coordinate']]))
traffic_df = traffic_df.drop_duplicates()
traffic_df['coordinates'] = traffic_df['coordinates'].apply(lambda x: str(x))

traffic_df['relativeSpeed'] = traffic_df.apply(lambda x: x['currentSpeed'] / x['freeFlowSpeed'], axis = 1)

today = datetime.datetime.now()
today = today.replace(second=0, microsecond=0, minute=0, hour=today.hour) + datetime.timedelta(hours=today.minute//30)
traffic_df['date']= [today for i in range(traffic_df.shape[0])]

traffic_df = traffic_df.drop(columns = ['@version'])

(177, 10)


,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,coordinates,relativeSpeed,date
0,FRC4,11,11,28,28,1.00,False,LINESTRING (18.038114420612857 59.342615187279...,1.000000,2024-12-18 20:00:00
1,FRC4,12,19,62,39,1.00,False,LINESTRING (18.036612383564147 59.342538726056...,0.631579,2024-12-18 20:00:00
2,FRC4,26,26,99,99,1.00,False,LINESTRING (18.060517571414948 59.338708520808...,1.000000,2024-12-18 20:00:00
3,FRC7,19,19,75,75,1.00,False,LINESTRING (18.0545509974599 59.34064910896488...,1.000000,2024-12-18 20:00:00
4,FRC4,13,21,153,95,0.99,False,LINESTRING (18.038586489399563 59.342495815378...,0.619048,2024-12-18 20:00:00


### 2.2. Collect weather data

In [11]:
weather_df = get_weather(coordinates)

,date,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
0,2024-12-18 20:00:00,6.126,-0.674,3.6,24.48,188.53627


### 2.3. Collect weather data

In [12]:
# CALENDAR_API_KEY = os.getenv('CALENDAR_API_KEY')
holiday_status = is_holiday(CALENDAR_API_KEY)
weather_df['holiday_status'] = holiday_status

,date,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,holiday_status
0,2024-12-18 20:00:00,6.126,-0.674,3.6,24.48,188.53627,0


### 2.4. Joining data and uploading to Hopsworks

In [ ]:
proj = hopsworks.login(project="ScalableMLandDeepLcourse")

In [16]:
# Initialize the feature store
fs = proj.get_feature_store()

# Define and insert the Forecast Weather Feature Group
feature_group_name = "stockholm_traffic"
version = 1
fg = fs.get_feature_group(name=feature_group_name, version=version)
fg.insert(traffic_df)

# Define and insert the Air Quality Feature Group
feature_group_name = "stockholm_weather_holiday"
version = 1
fg = fs.get_feature_group(name=feature_group_name, version=version)
fg.insert(weather_df)

Uploading Dataframe: 100.00% |██████████| Rows 177/177 | Elapsed Time: 00:01 | Remaining Time: 00:00


2024-12-18 20:10:38,020 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1170582/jobs/named/stockholm_traffic_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/ScalableMLandDeepLcourse/Resources/jobs/stockholm_traffic_1_offline_fg_materialization/config_1734547087388) to trigger the materialization job again.



Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


2024-12-18 20:10:43,715 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1170582/jobs/named/stockholm_weather_holiday_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/ScalableMLandDeepLcourse/Resources/jobs/stockholm_weather_holiday_1_offline_fg_materialization/config_1734547182650) to trigger the materialization job again.



(Job('stockholm_weather_holiday_1_offline_fg_materialization', 'SPARK'), None)